In [54]:
import os
import json
import glob
from concurrent.futures import ThreadPoolExecutor, as_completed
from inference_sdk import InferenceHTTPClient

client = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="4YTsQ9U6cmsFvVyPZuet"
)

def get_maltese_images(base_dir="."):
    image_paths = []
    for split in ["train", "valid", "test"]:
        folder = os.path.join(base_dir, split, "Pekinese")
        if os.path.exists(folder):
            for ext in ["*.jpg", "*.jpeg", "*.png", "*.bmp"]:
                image_paths.extend(glob.glob(os.path.join(folder, ext)))
    return image_paths

def process_single_image(img_path):
    try:
        result = client.run_workflow(
            workspace_name="classification-qqjwe",
            workflow_id="find-dogs-6",
            images={"image": img_path},
            use_cache=True
        )
        return {"path": img_path, "result": result, "error": None}
    except Exception as e:
        return {"path": img_path, "result": None, "error": str(e)}

def batch_process_images(image_paths, batch_size=20, output_file="Pekinese.json"):
    all_results = {}
    
    total_batches = (len(image_paths) + batch_size - 1) // batch_size
    
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]
        batch_num = i // batch_size + 1
        print(f"Обрабатываю batch {batch_num}/{total_batches}")
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_image = {executor.submit(process_single_image, img): img for img in batch}
            
            for future in as_completed(future_to_image):
                result = future.result()
                all_results[result["path"]] = {
                    "result": result["result"],
                    "error": result["error"]
                }
                
                if result["error"]:
                    print(f"Ошибка: {os.path.basename(result['path'])} - {result['error']}")
                else:
                    print(f"Успешно: {os.path.basename(result['path'])}")
    
    with open(output_file, 'w') as f:
        json.dump(all_results, f, indent=2, default=str)
    
    return all_results

if __name__ == "__main__":
    maltese_images = get_maltese_images()
    print(f"Найдено изображений Pekinese: {len(maltese_images)}")
    
    if maltese_images:
        results = batch_process_images(maltese_images, batch_size=100)
        print(f"\nОбработано {len(results)} изображений")

Найдено изображений Pekinese: 319
Обрабатываю batch 1/4
Успешно: Pekinese_10932_jpg.rf.01a368287347d86148c4e63d3aead5f3.jpg
Успешно: Pekinese_1020_jpg.rf.f104f989a055627643dfeed3f944ef76.jpg
Успешно: Pekinese_7817_jpg.rf.2c41b222b27e81d9a4086cdb7b60ef12.jpg
Успешно: Pekinese_19690_jpg.rf.2a8a1832eea37afa8b2abf64daefff1a.jpg
Успешно: Pekinese_3174_jpg.rf.a15ec6309995f217ef4f662400c46476.jpg
Успешно: Pekinese_10689_jpg.rf.520283127e512d0591ee34b48266850b.jpg
Успешно: Pekinese_5053_jpg.rf.3339be3261c239aa2ce31de9ceeac41c.jpg
Успешно: Pekinese_17291_jpg.rf.6650d55d54755b545157de02e50e8684.jpg
Успешно: Pekinese_6956_jpg.rf.b04f137d6e012bb7ecd459eca447173e.jpg
Успешно: Pekinese_1877_jpg.rf.4d76c33dddbe33ef188cf34a94277053.jpg
Успешно: Pekinese_937_jpg.rf.87fd7aebc53ec185dd4a2e005f9bead3.jpg
Успешно: Pekinese_2287_jpg.rf.549acc8172f1f12b4ddb81bb8706b28c.jpg
Успешно: Pekinese_5087_jpg.rf.b6578d9ff8eaf25999a6456670ff28fc.jpg
Успешно: Pekinese_1020_jpg.rf.bc4aa0210430383d58c84222a4e6e27f.jpg
Усп

In [56]:
import json
import numpy as np
import os
from pathlib import Path

def nms(boxes, scores, iou_threshold=0.5):
    """
    Non-Maximum Suppression из numpy
    boxes: [[x1, y1, x2, y2], ...] в абсолютных координатах
    scores: [confidence1, confidence2, ...]
    """
    if len(boxes) == 0:
        return []
    
    boxes = np.array(boxes)
    scores = np.array(scores)
    
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]
    
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        
        iou = inter / (areas[i] + areas[order[1:]] - inter)
        
        inds = np.where(iou <= iou_threshold)[0]
        order = order[inds + 1]
    
    return keep

def convert_to_x1y1x2y2(bbox):
    """Конвертирует [x_center, y_center, width, height] в [x1, y1, x2, y2]"""
    x, y, w, h = bbox
    x1 = x - w/2
    y1 = y - h/2
    x2 = x + w/2
    y2 = y + h/2
    return [x1, y1, x2, y2]

def calculate_centrality(x_center, y_center, img_width, img_height):
    """Вычисляет центральность (0-1, где 1 - центр)"""
    center_x = img_width / 2
    center_y = img_height / 2
    
    # Нормализованное расстояние от центра (0 = центр, 1 = угол)
    distance = np.sqrt(((x_center - center_x) / img_width)**2 + 
                       ((y_center - center_y) / img_height)**2)
    
    # Инвертируем: 1 - центр, 0 - край
    centrality = 1 - (distance * np.sqrt(2))  # sqrt(2) для нормализации
    return max(0, centrality)  # Не ниже 0

def select_best_bbox(predictions, img_width, img_height):
    """Выбирает лучший bounding box"""
    if not predictions:
        return None
    
    # 1. Фильтрация по размеру (7%-90%)
    filtered = []
    for pred in predictions:
        area = (pred['width'] * pred['height']) / (img_width * img_height)
        if 0.07 <= area <= 0.9:
            filtered.append(pred)
    
    # Если все отфильтровались, берем все
    if not filtered:
        filtered = predictions
    
    # Подготовка для NMS
    boxes = []
    scores = []
    bbox_data = []  # Сохраняем оригинальные данные
    
    for pred in filtered:
        bbox = [pred['x'], pred['y'], pred['width'], pred['height']]
        boxes.append(convert_to_x1y1x2y2(bbox))
        scores.append(pred['confidence'])
        bbox_data.append(pred)
    
    # 2. NMS
    if len(boxes) > 1:
        keep_indices = nms(boxes, scores, iou_threshold=0.5)
        filtered = [bbox_data[i] for i in keep_indices]
        boxes = [boxes[i] for i in keep_indices]
        scores = [scores[i] for i in keep_indices]
    
    if not filtered:
        return None
    
    # 3. Сортировка по confidence
    sorted_indices = np.argsort(scores)[::-1]
    filtered = [filtered[i] for i in sorted_indices]
    scores = [scores[i] for i in sorted_indices]
    
    # 4. Проверка центральности
    best_box = filtered[0]
    best_centrality = calculate_centrality(best_box['x'], best_box['y'], 
                                           img_width, img_height)
    
    if best_centrality > 0.3:  # Порог центральности
        return best_box
    
    # 5. Ищем более центральный бокс
    for i, pred in enumerate(filtered):
        centrality = calculate_centrality(pred['x'], pred['y'], 
                                         img_width, img_height)
        if centrality > best_centrality + 0.2:  # Существенно более центральный
            return pred
    
    return best_box  # Возвращаем самый уверенный

def process_json_file(input_file, output_file):
    """Обрабатывает JSON файл с аннотациями"""
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    processed_data = {}
    
    for img_path, content in data.items():
        if 'result' not in content or not content['result']:
            continue
        
        # Получаем информацию об изображении и предсказаниях
        result = content['result'][0]
        if 'predictions' not in result:
            continue
        
        predictions_data = result['predictions']
        img_width = predictions_data.get('image', {}).get('width', 256)
        img_height = predictions_data.get('image', {}).get('height', 256)
        predictions_list = predictions_data.get('predictions', [])
        
        # Выбираем лучший бокс
        best_box = select_best_bbox(predictions_list, img_width, img_height)
        
        # Создаем новую структуру
        if best_box:
            # Оставляем только нужные поля
            clean_box = {
                'x': best_box['x'],
                'y': best_box['y'],
                'width': best_box['width'],
                'height': best_box['height'],
                'confidence': best_box['confidence'],
                'class_id': best_box.get('class_id', 0),
                'class': best_box.get('class', 'dog')
            }
            
            processed_data[img_path] = {
                'result': [{
                    'predictions': {
                        'image': {'width': img_width, 'height': img_height},
                        'predictions': [clean_box]
                    }
                }]
            }
    
    # Сохраняем результат
    with open(output_file, 'w') as f:
        json.dump(processed_data, f, indent=2)
    
    print(f"Обработано: {len(processed_data)} изображений в {output_file}")
    return processed_data

# Обработка всех файлов
files = [
    ("Chihuahua.json", "Chihuahua_processed.json"),
    ("Japanese_spaniel.json", "Japanese_spaniel_processed.json"),
    ("Maltese_dog.json", "Maltese_dog_processed.json"),
    ("Pekinese.json", "Pekinese_processed.json")
]

for input_file, output_file in files:
    if os.path.exists(input_file):
        print(f"\nОбработка {input_file}...")
        process_json_file(input_file, output_file)
    else:
        print(f"Файл не найден: {input_file}")


Обработка Chihuahua.json...
Обработано: 320 изображений в Chihuahua_processed.json

Обработка Japanese_spaniel.json...
Обработано: 308 изображений в Japanese_spaniel_processed.json

Обработка Maltese_dog.json...
Обработано: 571 изображений в Maltese_dog_processed.json

Обработка Pekinese.json...
Обработано: 288 изображений в Pekinese_processed.json


In [46]:
import json
import os

def check_annotations_for_missing_boxes(annotation_file):
    """
    Проверяет, какие изображения не имеют bounding box в аннотациях
    """
    with open(annotation_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    images_without_boxes = []
    images_with_boxes = []
    total_count = len(data)
    
    for img_path, content in data.items():
        has_box = False
        
        # Проверяем наличие result
        if "result" in content and content["result"]:
            for result_item in content["result"]:
                # Проверяем наличие predictions
                if "predictions" in result_item:
                    predictions_data = result_item["predictions"]
                    
                    # Проверяем наличие списка predictions внутри predictions
                    if ("predictions" in predictions_data and 
                        isinstance(predictions_data["predictions"], list) and 
                        len(predictions_data["predictions"]) > 0):
                        
                        # Проверяем, есть ли хоть один валидный бокс
                        for pred in predictions_data["predictions"]:
                            if all(key in pred for key in ["x", "y", "width", "height"]):
                                has_box = True
                                break
        
        if has_box:
            images_with_boxes.append(img_path)
        else:
            images_without_boxes.append(img_path)
    
    return {
        "total": total_count,
        "with_boxes": len(images_with_boxes),
        "without_boxes": len(images_without_boxes),
        "percentage_with_boxes": (len(images_with_boxes) / total_count * 100) if total_count > 0 else 0,
        "images_without_boxes": images_without_boxes,
        "images_with_boxes": images_with_boxes
    }

def analyze_all_annotation_files(files):
    """
    Анализирует все файлы аннотаций и выводит сводку
    """
    results = {}
    
    for file in files:
        if os.path.exists(file):
            print(f"\n{'='*60}")
            print(f"Анализ файла: {file}")
            print('='*60)
            
            result = check_annotations_for_missing_boxes(file)
            results[file] = result
            
            print(f"Всего изображений: {result['total']}")
            print(f"С боксами: {result['with_boxes']}")
            print(f"Без боксов: {result['without_boxes']}")
            print(f"Процент с боксами: {result['percentage_with_boxes']:.1f}%")
            
            if result['without_boxes'] > 0:
                print(f"\nИзображения без боксов ({len(result['images_without_boxes'])}):")
                for i, img_path in enumerate(result['images_without_boxes'][:10]):  # Показываем первые 10
                    print(f"  {i+1}. {os.path.basename(img_path)}")
                if len(result['images_without_boxes']) > 10:
                    print(f"  ... и еще {len(result['images_without_boxes']) - 10} изображений")
        else:
            print(f"\nФайл не найден: {file}")
    
    return results

def generate_summary_report(results):
    """
    Генерирует сводный отчет по всем файлам
    """
    print(f"\n{'='*60}")
    print("СВОДНЫЙ ОТЧЕТ")
    print('='*60)
    
    total_all_images = 0
    total_with_boxes = 0
    total_without_boxes = 0
    
    for file_name, data in results.items():
        total_all_images += data['total']
        total_with_boxes += data['with_boxes']
        total_without_boxes += data['without_boxes']
        
        print(f"\n{os.path.basename(file_name)}:")
        print(f"  Изображений с боксами: {data['with_boxes']}/{data['total']} ({data['percentage_with_boxes']:.1f}%)")
    
    if total_all_images > 0:
        percentage_total = (total_with_boxes / total_all_images) * 100
        print(f"\n{'='*40}")
        print(f"ИТОГО по всем файлам:")
        print(f"  Всего изображений: {total_all_images}")
        print(f"  С боксами: {total_with_boxes} ({percentage_total:.1f}%)")
        print(f"  Без боксов: {total_without_boxes} ({100 - percentage_total:.1f}%)")
    
    # Сохраняем отчет в файл
    with open("annotation_report.txt", "w", encoding='utf-8') as f:
        f.write("ОТЧЕТ ПО АННОТАЦИЯМ\n")
        f.write("="*50 + "\n\n")
        
        for file_name, data in results.items():
            f.write(f"Файл: {os.path.basename(file_name)}\n")
            f.write(f"  Всего изображений: {data['total']}\n")
            f.write(f"  С боксами: {data['with_boxes']} ({data['percentage_with_boxes']:.1f}%)\n")
            f.write(f"  Без боксов: {data['without_boxes']}\n")
            
            if data['without_boxes'] > 0:
                f.write("\n  Изображения без боксов:\n")
                for img_path in data['images_without_boxes']:
                    f.write(f"    - {os.path.basename(img_path)}\n")
            f.write("\n")
        
        if total_all_images > 0:
            f.write("="*50 + "\n")
            f.write(f"ИТОГО:\n")
            f.write(f"  Всего изображений: {total_all_images}\n")
            f.write(f"  С боксами: {total_with_boxes} ({(total_with_boxes/total_all_images*100):.1f}%)\n")
            f.write(f"  Без боксов: {total_without_boxes}\n")
    
    print(f"\nПолный отчет сохранен в: annotation_report.txt")

# Список файлов для проверки
annotation_files = [
    "Chihuahua.json",
    "Japanese_spaniel.json", 
    "Maltese_dog.json",
    "Pekinese.json"
]

# Проверяем наличие альтернативных имен файлов
alternative_files = [
    "chihuahua_annotations.json",
    "japanese_spaniel_annotations.json",
    "maltese_annotations.json", 
    "pekinese_annotations.json"
]

# Сначала проверяем основные имена, если нет - альтернативные
files_to_check = []
for main_file, alt_file in zip(annotation_files, alternative_files):
    if os.path.exists(main_file):
        files_to_check.append(main_file)
    elif os.path.exists(alt_file):
        files_to_check.append(alt_file)
        print(f"Используется альтернативное имя: {alt_file}")
    else:
        print(f"Не найден ни один из файлов: {main_file} или {alt_file}")

if files_to_check:
    # Анализируем все файлы
    results = analyze_all_annotation_files(files_to_check)
    
    # Генерируем сводный отчет
    generate_summary_report(results)
    
    # Дополнительная проверка: найти дубликаты между файлами
    print(f"\n{'='*60}")
    print("ПРОВЕРКА НА ДУБЛИКАТЫ МЕЖДУ ФАЙЛАМИ")
    print('='*60)
    
    all_image_paths = []
    for file_name, data in results.items():
        all_image_paths.extend(data['images_with_boxes'])
        all_image_paths.extend(data['images_without_boxes'])
    
    from collections import Counter
    duplicates = {k: v for k, v in Counter(all_image_paths).items() if v > 1}
    
    if duplicates:
        print(f"Найдено дубликатов: {len(duplicates)}")
        for img_path, count in list(duplicates.items())[:5]:
            print(f"  {img_path}: встречается {count} раз(а)")
    else:
        print("Дубликатов не найдено")
else:
    print("Нет файлов для анализа. Проверьте наличие аннотационных файлов.")


Анализ файла: Chihuahua.json
Всего изображений: 323
С боксами: 320
Без боксов: 3
Процент с боксами: 99.1%

Изображения без боксов (3):
  1. Chihuahua_575_jpg.rf.797af49490645a79b36b1c8f62d62938.jpg
  2. Chihuahua_3651_jpg.rf.93ea2b458518b72d3308d72e87cc2f64.jpg
  3. Chihuahua_13383_jpg.rf.58bac7adbf2c934cd62d619a389b386f.jpg

Анализ файла: Japanese_spaniel.json
Всего изображений: 397
С боксами: 308
Без боксов: 89
Процент с боксами: 77.6%

Изображения без боксов (89):
  1. Japanese_spaniel_3720_jpg.rf.fbe5fd8f396202bdbae52b6bb6144034.jpg
  2. Japanese_spaniel_3052_jpg.rf.5e577aba9e96cdecfd26ceebcf95d8c1.jpg
  3. Japanese_spaniel_1890_jpg.rf.588f94d0ecb1471d0018f7952f12918e.jpg
  4. Japanese_spaniel_3781_jpg.rf.df91c4b565aa27fb047ef7d65c81c916.jpg
  5. Japanese_spaniel_2886_jpg.rf.602e7f3fb33c30e519fbd771395451d1.jpg
  6. Japanese_spaniel_4511_jpg.rf.a3b7a407e06491b3124d8c13c4321fd9.jpg
  7. Japanese_spaniel_1724_jpg.rf.90e8491d8ff20f26aaf9ff7b76e9a3c1.jpg
  8. Japanese_spaniel_2162_jpg

In [76]:
import json
import os
import glob
from pathlib import Path

# Маппинг пород к классам
BREED_TO_CLASS = {
    "Chihuahua": 0,
    "Japanese_spaniel": 1,
    "Maltese_dog": 2,
    "Pekinese": 3
}

def convert_json_to_yolo(json_file, base_dir="."):
    """
    Конвертирует JSON аннотации в формат YOLO (.txt файлы)
    """
    breed_name = Path(json_file).stem.replace("_processed", "")
    
    if breed_name not in BREED_TO_CLASS:
        print(f"Неизвестная порода: {breed_name}")
        return
    
    class_id = BREED_TO_CLASS[breed_name]
    
    # Загружаем JSON файл
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    processed_count = 0
    missing_count = 0
    
    for img_path, content in data.items():
        # Получаем информацию об изображении
        if "result" not in content or not content["result"]:
            missing_count += 1
            continue
        
        result_item = content["result"][0]
        if "predictions" not in result_item:
            missing_count += 1
            continue
        
        predictions = result_item["predictions"]
        image_info = predictions.get("image", {})
        bbox_list = predictions.get("predictions", [])
        
        if not bbox_list:
            missing_count += 1
            continue
        
        # Берем первый бокс (самый лучший после фильтрации)
        bbox = bbox_list[0]
        
        # Получаем путь к изображению из JSON ключа
        # JSON ключ выглядит как: "./train/Chihuahua/Chihuahua_1271_jpg.rf.cb99fc6d7368e8530b0fa0b76100bc8e.jpg"
        # Нужно извлечь split (train/valid/test), breed и имя файла
        parts = img_path.strip("./").split("/")
        if len(parts) < 3:
            print(f"Некорректный путь: {img_path}")
            continue
        
        split = parts[0]  # train, valid или test
        breed = parts[1]  # порода (должна совпадать с breed_name)
        filename = parts[2]
        
        # Проверяем, существует ли изображение в новой структуре
        # Ищем изображение в папке images
        image_dir = os.path.join(base_dir, split, breed, "images")
        image_path = os.path.join(image_dir, filename)
        
        if not os.path.exists(image_path):
            # Пробуем найти файл с другим расширением
            found = False
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                alt_path = os.path.join(image_dir, os.path.splitext(filename)[0] + ext)
                if os.path.exists(alt_path):
                    image_path = alt_path
                    filename = os.path.basename(alt_path)
                    found = True
                    break
            
            if not found:
                missing_count += 1
                continue
        
        # Получаем размеры изображения
        img_width = image_info.get("width", 256)
        img_height = image_info.get("height", 256)
        
        # Координаты бокса
        x_center = bbox.get("x", 0)
        y_center = bbox.get("y", 0)
        width = bbox.get("width", 0)
        height = bbox.get("height", 0)
        
        # Нормализуем координаты (0-1)
        x_norm = x_center / img_width
        y_norm = y_center / img_height
        w_norm = width / img_width
        h_norm = height / img_height
        
        # Создаем строку для YOLO формата
        yolo_line = f"{class_id} {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}"
        
        # Создаем путь к файлу аннотаций
        label_dir = os.path.join(base_dir, split, breed, "labels")
        label_filename = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(label_dir, label_filename)
        
        # Записываем аннотацию
        os.makedirs(label_dir, exist_ok=True)
        with open(label_path, 'w', encoding='utf-8') as f:
            f.write(yolo_line)
        
        processed_count += 1
    
    print(f"Обработано {breed_name}: {processed_count} аннотаций, пропущено: {missing_count}")

def process_all_json_files():
    """
    Обрабатывает все обработанные JSON файлы
    """
    json_files = [
        "Chihuahua_processed.json",
        "Japanese_spaniel_processed.json",
        "Maltese_dog_processed.json",
        "Pekinese_processed.json"
    ]
    
    for json_file in json_files:
        if os.path.exists(json_file):
            print(f"\nКонвертация {json_file}...")
            convert_json_to_yolo(json_file)
        else:
            print(f"Файл не найден: {json_file}")

def check_labels_structure():
    """
    Проверяет структуру папок labels и подсчитывает аннотации
    """
    splits = ['train', 'valid', 'test']
    breeds = ['Chihuahua', 'Japanese_spaniel', 'Maltese_dog', 'Pekinese']
    
    print("\nПроверка структуры аннотаций:")
    print("-" * 50)
    
    for split in splits:
        for breed in breeds:
            labels_dir = os.path.join(split, breed, "labels")
            images_dir = os.path.join(split, breed, "images")
            
            if os.path.exists(labels_dir):
                label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
                image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))] if os.path.exists(images_dir) else []
                
                print(f"{split}/{breed}:")
                print(f"  Аннотации: {len(label_files)} файлов")
                print(f"  Изображения: {len(image_files)} файлов")
                
                # Проверяем соответствие имен
                if label_files and image_files:
                    missing_labels = []
                    for img_file in image_files:
                        label_file = os.path.splitext(img_file)[0] + ".txt"
                        if label_file not in label_files:
                            missing_labels.append(img_file)
                    
                    if missing_labels:
                        print(f"  Отсутствуют аннотации для {len(missing_labels)} изображений")

def create_yaml_config():
    """
    Создает YAML конфиг для YOLO
    """
    yaml_content = """# YOLO dataset configuration
path: .  # dataset root dir

# Train/val/test splits
train: train  # training images (relative to 'path')
val: valid    # validation images (relative to 'path')
test: test    # test images (relative to 'path')

# Classes
names:
  0: Chihuahua
  1: Japanese_spaniel
  2: Maltese_dog
  3: Pekinese

nc: 4  # number of classes
"""
    
    with open("dataset.yaml", "w", encoding="utf-8") as f:
        f.write(yaml_content)
    
    print("\nСоздан файл конфигурации: dataset.yaml")

if __name__ == "__main__":
    print("Конвертация JSON аннотаций в формат YOLO...")
    print("=" * 60)
    
    # Конвертируем JSON файлы в YOLO формат
    process_all_json_files()
    
    # Проверяем структуру
    check_labels_structure()
    
    # Создаем YAML конфиг
    create_yaml_config()
    
    print("\n" + "=" * 60)
    print("Конвертация завершена!")
    print("Для тренировки YOLO используйте файл: dataset.yaml")

Конвертация JSON аннотаций в формат YOLO...

Конвертация Chihuahua_processed.json...
Обработано Chihuahua: 320 аннотаций, пропущено: 0

Конвертация Japanese_spaniel_processed.json...
Обработано Japanese_spaniel: 308 аннотаций, пропущено: 0

Конвертация Maltese_dog_processed.json...
Обработано Maltese_dog: 571 аннотаций, пропущено: 0

Конвертация Pekinese_processed.json...
Обработано Pekinese: 288 аннотаций, пропущено: 0

Проверка структуры аннотаций:
--------------------------------------------------
train/Chihuahua:
  Аннотации: 252 файлов
  Изображения: 252 файлов
train/Japanese_spaniel:
  Аннотации: 274 файлов
  Изображения: 274 файлов
train/Maltese_dog:
  Аннотации: 479 файлов
  Изображения: 479 файлов
train/Pekinese:
  Аннотации: 242 файлов
  Изображения: 242 файлов
valid/Chihuahua:
  Аннотации: 27 файлов
  Изображения: 27 файлов
valid/Japanese_spaniel:
  Аннотации: 43 файлов
  Изображения: 43 файлов
valid/Maltese_dog:
  Аннотации: 46 файлов
  Изображения: 46 файлов
valid/Pekinese

In [80]:
import os
from pathlib import Path

dataset_path = Path(".")
for split in ['train', 'valid', 'test']:
    for class_dir in (dataset_path / split).iterdir():
        img_dir = class_dir / 'images'
        lbl_dir = class_dir / 'labels'
        imgs = set(f.stem for f in img_dir.glob('*.jpg'))
        lbls = set(f.stem for f in lbl_dir.glob('*.txt'))
        missing = imgs - lbls
        if missing:
            print(f"ВНИМАНИЕ: В {class_dir} отсутствуют аннотации для {len(missing)} изображений.")